In [1]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.10.1
True


In [6]:
%%writefile configs/resnet/resnet18_alzheimer_coronal_view.py
_base_ = [
    '../_base_/models/resnet18.py', 
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    '../_base_/default_runtime.py'
]

pretrained='https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_b16x8_cifar10_20210528-bd6371c8.pth'

model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet', 
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=pretrained,
            prefix='backbone'
        ),
    ),
    head=dict(
        num_classes=3,
        topk=(1, 3),
    ))

dataset_type = 'CustomDataset'
# dataset_type = 'ImageNet'
data_preprocessor = dict(
     # mean=[124.508, 116.050, 106.438],
     # std=[58.577, 57.310, 57.437],
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    # dict(type='RandomResizedCrop', scale=224), # 224x224
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    # dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    # dict(type='CenterCrop', crop_size=224),
    dict(type='PackInputs'),    
]

train_dataloader = dict(
    batch_size=32,
    num_workers=4,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal/train',
        ann_file='Coronal/train_ann.txt',
        classes='data/classes.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=32,
    num_workers=4,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal/validation',
        classes='data/classes.txt',
        ann_file='Coronal/val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=32,
    num_workers=4,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal/test',
        classes='data/classes.txt',
        ann_file='Coronal/test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))
test_evaluator = val_evaluator 

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=128)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Overwriting configs/resnet/resnet18_alzheimer_coronal_view.py


In [5]:
!python ./tools/train.py \
    configs/resnet/resnet18_alzheimer_coronal_view.py \
    --work-dir work_dirs/alzheimer/coronal/resnet18 \
    # --auto-scale-lr

configs/resnet/resnet18_alzheimer_coronal_view.py
11/21 00:46:38 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1836788448
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode

In [12]:
import os
import subprocess
train_AD = 'data/Coronal/train/AD'
train_MCI = 'data/Coronal/train/MCI'
train_NC = 'data/Coronal/train/NC'

val_AD = 'data/Coronal/validation/AD'
val_MCI = 'data/Coronal/validation/MCI'
val_NC = 'data/Coronal/validation/NC'

test_AD = 'data/Coronal/test/AD'
test_MCI = 'data/Coronal/test/MCI'
test_NC = 'data/Coronal/test/NC'


trainheat_AD = 'data/resnet_heat_coronal/train/AD'
trainheat_MCI = 'data/resnet_heat_coronal/train/MCI'
trainheat_NC = 'data/resnet_heat_coronal/train/NC'

valheat_AD = 'data/resnet_heat_coronal/validation/AD'
valheat_MCI = 'data/resnet_heat_coronal/validation/MCI'
valheat_NC = 'data/resnet_heat_coronal/validation/NC'

count = 0
for i in os.listdir(val_AD):
    count+=1
    temp = os.path.join(val_AD,i)
    out = valheat_AD + '/heat_'+i
    print(out)  
    subprocess.run(["python", "tools/visualization/vis_cam.py", 
                temp,
                 "configs/resnet/resnet18_alzheimer_coronal_view.py",
                 "work_dirs/alzheimer/coronal/resnet18/epoch_100.pth",
                 # "--target-layers", "backbone.layer4.1.bn2",
                 "--method", "GradCAM++",
                 "--save-path", out,
                 # "--eigen-smooth", "--aug-smooth"
                 ], text=True)
    print(count)
    # break

data/resnet_heat_coronal/validation/AD/heat_y-106 (5).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/coronal/resnet18/epoch_100.pth
20
Automatically choose the last norm layer "backbone.layer4.1.bn2" as the target layer.
1
data/resnet_heat_coronal/validation/AD/heat_y-138 (4).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/coronal/resnet18/epoch_100.pth
20
Automatically choose the last norm layer "backbone.layer4.1.bn2" as the target layer.
2
data/resnet_heat_coronal/validation/AD/heat_y-101.png
Loads checkpoint by local backend from path: work_dirs/alzheimer/coronal/resnet18/epoch_100.pth
20
Automatically choose the last norm layer "backbone.layer4.1.bn2" as the target layer.
3
data/resnet_heat_coronal/validation/AD/heat_y-103 (6).png
Loads checkpoint by local backend from path: work_dirs/alzheimer/coronal/resnet18/epoch_100.pth
20
Automatically choose the last norm layer "backbone.layer4.1.bn2" as the target layer.
4
data/resnet_heat_coronal

In [13]:
%%writefile configs/resnet/resnet18_alzheimer_coronal_view_heat.py
_base_ = [
    '../_base_/models/resnet18.py', 
    '../_base_/schedules/imagenet_bs1024_adamw_conformer.py',
    '../_base_/default_runtime.py'
]

pretrained='https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_b16x8_cifar10_20210528-bd6371c8.pth'

model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet', 
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=pretrained,
            prefix='backbone'
        ),
    ),
    head=dict(
        num_classes=3,
        topk=(1, 3),
    ))

dataset_type = 'CustomDataset'
# dataset_type = 'ImageNet'
data_preprocessor = dict(
     # mean=[124.508, 116.050, 106.438],
     # std=[58.577, 57.310, 57.437],
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    dict(type='RandomResizedCrop', scale=224), # 224x224
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),   # random horizontal flip
    dict(type='PackInputs'),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),     # read image
    dict(type='ResizeEdge', scale=256),  # Scale the short side to 256
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackInputs'),    
]

train_dataloader = dict(
    batch_size=32,
    num_workers=6,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal_resnet_heat/train',
        # ann_file='Coronal/train_ann.txt',
        classes='data/classes.txt',
        with_label=True,
        pipeline=train_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=True),
    persistent_workers=True
)

val_dataloader = dict(
    batch_size=32,
    num_workers=6,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal_resnet_heat/validation',
        classes='data/classes.txt',
        # ann_file='Coronal/val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

test_dataloader = dict(
    batch_size=32,
    num_workers=6,
    dataset=dict(
        type=dataset_type,
        data_root='data',
        data_prefix='Coronal_resnet_heat/test',
        classes='data/classes.txt',
        # ann_file='Coronal/test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
    sampler=dict(type='DefaultSampler', shuffle=False),
    persistent_workers=True
)

val_evaluator = dict(type='Accuracy', topk=(1, 3))
test_evaluator = val_evaluator 

optim_wrapper = dict(
    optimizer=dict(
        type='AdamW',
        # for batch in each gpu is 128, 8 gpu
        # lr = 5e-4 * 128 * 8 / 512 = 0.001
        lr=5e-4 * 128 * 8 / 512,
        weight_decay=0.05,
        eps=1e-8,
        betas=(0.9, 0.999)),
    paramwise_cfg=dict(
        norm_decay_mult=0.0,
        bias_decay_mult=0.0,
        custom_keys={
            '.cls_token': dict(decay_mult=0.0),
        }),
)

# learning policy
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1e-3,
        by_epoch=True,
        begin=0,
        end=5,
        convert_to_iter_based=True),
    dict(
        type='CosineAnnealingLR',
        T_max=295,
        eta_min=1e-5,
        by_epoch=True,
        begin=5,
        end=300)
]

train_cfg = dict(by_epoch=True, max_epochs=100, val_interval=1)
val_cfg = dict()
test_cfg = dict()

auto_scale_lr = dict(base_batch_size=256)

default_scope = 'mmpretrain'

# configure default hooks
default_hooks = dict(
    # record the time of every iteration.
    timer=dict(type='IterTimerHook'),

    # print log every 100 iterations.
    logger=dict(type='LoggerHook', interval=100),

    # enable the parameter scheduler.
    param_scheduler=dict(type='ParamSchedulerHook'),

    # save checkpoint per epoch.
    checkpoint=dict(type='CheckpointHook', interval=1),

    # set sampler seed in a distributed environment.
    sampler_seed=dict(type='DistSamplerSeedHook'),

    # validation results visualization, set True to enable it.
    visualization=dict(type='VisualizationHook', enable=False),
)

env_cfg = dict(
    # whether to enable cudnn benchmark
    cudnn_benchmark=False,

    # set multi-process parameters
    mp_cfg=dict(mp_start_method='fork', opencv_num_threads=0),

    # set distributed parameters
    dist_cfg=dict(backend='nccl'),
)

# set visualizer
vis_backends = [dict(type='LocalVisBackend')]  # use local HDD backend
visualizer = dict(type='UniversalVisualizer', vis_backends=vis_backends, name='visualizer')

# set log level
log_level = 'INFO'

# load from which checkpoint
load_from = None

# whether to resume training from the loaded checkpoint
resume = False

Writing configs/resnet/resnet18_alzheimer_coronal_view_heat.py


In [14]:
!python ./tools/train.py \
    configs/resnet/resnet18_alzheimer_coronal_view_heat.py \
    --work-dir work_dirs/alzheimer/coronal/resnet18_heat \
    # --auto-scale-lr

configs/resnet/resnet18_alzheimer_coronal_view_heat.py
11/22 20:18:16 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 (default, Sep 11 2023, 13:40:15) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1540065216
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda-11.4
    NVCC: Cuda compilation tools, release 11.4, V11.4.100
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.10.1
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -ge